# Install Pre-requisites

In [2]:
!pip install -i https://test.pypi.org/simple/ arangomlFeatureStore
!pip install  pyArango python-arango PyYAML==5.2

Looking in indexes: https://test.pypi.org/simple/
     |████████████████████████████████| 50 kB 3.0 MB/s 
     |████████████████████████████████| 95 kB 3.2 MB/s 
     |████████████████████████████████| 265 kB 45.9 MB/s 
     |████████████████████████████████| 60 kB 6.6 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 251 kB 55.9 MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.2-cp37-cp37m-linux_x86_64.whl size=44225 sha256=f195cb75e8738a8b11c87bca053bbe834ab1c26af90635dc6178d14667b7705a
  Stored in directory: /root/.cache/pip/wheels/c5/67/65/80b02387630c3794fc9bab68ca0de3dd87d639d65936026527
  Created wheel for pyArango: filename=pyArango-2.0.1-py2.py3-none-any.whl size=52259 sha256=5229991a73c0984edb51f9e9af67471e084239d80240ceca0ab4b43b044a66fe
  Stored in direct

# Add Package Path to Search Path 

In [3]:
import arangomlFeatureStore as p
print(p.__path__)

['/usr/local/lib/python3.7/dist-packages/arangomlFeatureStore']


In [4]:
import sys
sys.path.append(p.__path__)

In [5]:
!chmod -R 777 /usr/local/lib/python3.7/dist-packages/arangomlFeatureStore

In [9]:
!chmod -R 777 /usr/local/lib/python3.7/dist-packages/arangomlFeatureStore

In [10]:
import sys
sys.path.insert(0, "/usr/local/lib/python3.7/dist-packages/arangomlFeatureStore")

# Interaction


1.   Producer applications:
Register/add entities with the feature store
Create embeddings for them
Associate entities with feature values. The association is identified by a tag
2. Consumer applications:
  Can search for a set of features that match a tag property
  Can lookup/search for an entity by property


Usage scenario:
Producer applications create embeddings and store them in the feature store. Consumer applications consume embeddings by specifying a featureset tag.

## Connect to Oasis and get a Database

In [6]:
from arangomlFeatureStore.feature_store_admin import FeatureStoreAdmin
from arango.database import StandardDatabase

In [7]:
fa = FeatureStoreAdmin()

Credentials expired.
Requesting new temp credentials.
Temp database ready to use.


In [11]:
fa = FeatureStoreAdmin()
db = fa.db

Requesting new temp credentials.
Temp database ready to use.


In [12]:
import uuid

## Add an Entity

In [13]:
fs = fa.get_feature_store()
rk = str(uuid.uuid4())
test_entity = {'_key': 'test_entity' + rk , 'attribute': 'some value'}
entity_info = fs.add_entity(test_entity)
cfg = fa.cfg
ENTITY_COLL = cfg['arangodb']['entity_col']
assert entity_info['_id'] == ENTITY_COLL + '/' + ('test_entity' + rk)
assert entity_info['_key'] == ('test_entity' + rk)


## Add a Feature Value

In [14]:
rk2 = str(uuid.uuid4())
test_value = {'_key': 'test_value' + rk2, 'attribute': [0.1234, 0.1234, 0.1234]}
value_info = fs.add_value(test_value)
cfg = fa.cfg
FEATURE_VAL_COLL = cfg['arangodb']['feature_value_col']
assert value_info['_id'] == FEATURE_VAL_COLL + '/' + ('test_value' + rk2)
assert value_info['_key'] == ('test_value' + rk2)

## Associate a Feature Value with an Entity
**Note**: Note the tag attribute specified

In [15]:
edoc = {'_from': entity_info['_key'],'_to': value_info['_key'], 'tag': 'model-xyz'}
edge_info = fs.link_entity_feature_value(edoc)
EDGE_COLL = cfg['arangodb']['edge_col']
# edge_id = EDGE_COLL + '/' + edge_info['_key'] + '-' + value_info['_key']
# assert edge_info['_id'] == edge_id


## Lookup/Search for an Entity by Tag

In [16]:
fs.find_entity(attrib_name='_key', attrib_value= "test_entity" + rk)

[{'_id': 'entity/test_entityf30fa925-f881-4faf-8daf-4c2e27099dd6',
  '_key': 'test_entityf30fa925-f881-4faf-8daf-4c2e27099dd6',
  '_rev': '_dn82Nti---',
  'attribute': 'some value'}]

## Add a Dataset with a Tag attribute to the Feature Store

In [17]:
DATASET_SIZE = 10
EDGE_COLL = cfg['arangodb']['edge_col']
for i in range(DATASET_SIZE):
  rkd = str(uuid.uuid4())
  test_entity_ds = {'_key': 'test_entity' + rkd , 'attribute': 'some value'}
  entity_info_ds = fs.add_entity(test_entity_ds)
  rkd2 = str(uuid.uuid4())
  test_value_ds = {'_key': 'test_value' + rkd2, 'attribute': [0.1234, 0.1234, 0.1234]}
  value_info_ds = fs.add_value(test_value_ds)
  edoc_ds = {'_from': entity_info_ds['_key'],'_to': value_info_ds['_key'], 'tag': 'model-xyz'}
  edge_info_ds = fs.link_entity_feature_value(edoc_ds)
  print(f"Added doc # {(i+1)} to feature store with tag: model-xyz")



Added doc # 1 to feature store with tag: model-xyz
Added doc # 2 to feature store with tag: model-xyz
Added doc # 3 to feature store with tag: model-xyz
Added doc # 4 to feature store with tag: model-xyz
Added doc # 5 to feature store with tag: model-xyz
Added doc # 6 to feature store with tag: model-xyz
Added doc # 7 to feature store with tag: model-xyz
Added doc # 8 to feature store with tag: model-xyz
Added doc # 9 to feature store with tag: model-xyz
Added doc # 10 to feature store with tag: model-xyz


## Retrieve a FeatureSet with a specified Tag Property Value

In [18]:
fs.get_featureset_with_tag('tag', 'model-xyz')

[{'_id': 'feature-value/test_value86d54909-3c2b-487b-91a8-fde70a26630f',
  '_key': 'test_value86d54909-3c2b-487b-91a8-fde70a26630f',
  '_rev': '_dn82Oe6---',
  'attribute': [0.1234, 0.1234, 0.1234]},
 {'_id': 'feature-value/test_value95221985-0413-468e-bc21-793c3565d81b',
  '_key': 'test_value95221985-0413-468e-bc21-793c3565d81b',
  '_rev': '_dn82PB---_',
  'attribute': [0.1234, 0.1234, 0.1234]},
 {'_id': 'feature-value/test_valuecaa5447e-3f34-4421-86a1-b7649bcbba76',
  '_key': 'test_valuecaa5447e-3f34-4421-86a1-b7649bcbba76',
  '_rev': '_dn82PNu--A',
  'attribute': [0.1234, 0.1234, 0.1234]},
 {'_id': 'feature-value/test_value95b2cd9a-4c6e-43b4-b431-b29611a3a43e',
  '_key': 'test_value95b2cd9a-4c6e-43b4-b431-b29611a3a43e',
  '_rev': '_dn82Pam--_',
  'attribute': [0.1234, 0.1234, 0.1234]},
 {'_id': 'feature-value/test_value6811966f-1429-42af-a2cf-e10859835426',
  '_key': 'test_value6811966f-1429-42af-a2cf-e10859835426',
  '_rev': '_dn82Pp2--_',
  'attribute': [0.1234, 0.1234, 0.1234]},
